In [ ]:
# importing the module
import cv2
import pandas as pd
import numpy as np
import random as rnd
import os
from PIL import Image
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import shutil


In [ ]:

def calc_mean(frame, toplx, toply , wid, heig):
    ######
    # function to calculate the mean color inside a box 
    ######
    # frame: number of the fram for find the file in the pc
    # toplx: top left corner x coordinate of the box
    # toply: top left corner y coordinate of the box
    # wid: width of the box
    # heig: height of the box
    ######

    eps = 3 # width of the frame to ignore

    xA, xB = toplx + eps , toplx + wid - eps  # find the coordinate of the corners
    if xA >= xB : xB = xA + eps # negative length check
    yA, yB = toply + eps , toply + heig - eps 
    if yA >= yB : yB = yA + eps

    path_imm = '/media/alb/Elements/DATASET/SoccerNet/tracking/input/img1/{}.jpg'.format(str(frame).zfill(6))
    im = Image.open(path_imm)
    im_array = np.array(im)
    #print('shape: ',im_array.shape)
    #print('xA, xB, yA, yB\n', xA, xB, yA, yB)
    #print(im_array[xA:xB,yA:yB,0])
    lev3_mean = np.array([np.round(np.mean(im_array[yA:yB,xA:xB,i])) for i in range(im_array.shape[2])]) # mean compute

    return lev3_mean



def mod_dataset(det: pd.DataFrame , gt: pd.DataFrame, i=0):
    ######
    # function to calculate all the modification to the dataset
    ######
    # det: table det in the original dataset
    # gt: table gt in the original dataset
    # i: index (not used in this version)
    ######


    gt['area']= gt.width * gt.height # adding area column
    gt['dim_sum']= np.abs(gt.width + gt.height) # adding sum of dimensions column
    gt['dim_diff']= np.abs(gt.width - gt.height) # adding difference of dimensions column
    gt_track_id = gt.groupby('track_id').mean() # grouping by track id to find the mean dim of each entity
    gt_count_frame = gt.groupby('track_id').count() # groping by track id to find the most frequent id

    # aggiungere un check di sicurezza ?
    dim_ok = gt_track_id[(gt_track_id.dim_sum < 50) & (gt_track_id.dim_diff < 5)] # filter for dimension, should be almost a square


    #res=gt_count_frame[gt_count_frame['acc'] == gt_count_frame.max().acc] # indice della palla
    res = dim_ok

    list_frames = gt[gt.track_id == res.index[0]].frame # finding the track id of ball

    for frame in list_frames:
        actual_frame = gt[(gt.track_id == res.index[0]) & (gt.frame == frame)] # la riga effettiva delle informazioni sul frame e l'indice
        gt.loc[(gt.width == actual_frame.width.values[0]) & (gt.frame == frame) &(gt.x_cord == actual_frame.x_cord.values[0]), 'ball_check'] = 1 # changing the value in the det table
        det.loc[(det.frame == frame)&(det.width == actual_frame.width.values[0]) &(det.x_cord == actual_frame.x_cord.values[0]), 'ball_check'] = 1  # changing the value in the gt table

    # computing labels for teams
    new_c = det.apply(lambda row : calc_mean(row['frame'], row['x_cord'], row['y_cord'], row['width'], row['height']), axis=1)
    X = np.array([list(a) for a in new_c.values])
    gm = GaussianMixture(n_components=3, random_state=0).fit(X)
    det['team_label'] = gm.predict(X)

    gt.frame.map(lambda x: x + i*750)
    det.frame.map(lambda x: x + i*750)

    gt = gt.drop(['area', 'dim_sum', 'dim_diff', 'unk2', 'unk3'], axis=1)
    det = det.drop(['unk2', 'unk3'], axis=1)
    
    return det, gt 


In [ ]:
input_path_det = '/media/alb/Elements/DATASET/SoccerNet/tracking/input/det/det.txt'
input_path_gt = '/media/alb/Elements/DATASET/SoccerNet/tracking/input/gt/gt.txt'
input_path_img = '/media/alb/Elements/DATASET/SoccerNet/tracking/input/img1'



input_gt = pd.read_csv(input_path_gt, header= None, names=['frame', 'track_id', 'x_cord', 'y_cord', 'width', 'height', 'acc', 'unk2', 'unk3', 'ball_check'], index_col=False)
input_det = pd.read_csv(input_path_det, header= None, names=['frame', 'unk1', 'x_cord', 'y_cord', 'width', 'height', 'acc', 'unk2', 'unk3', 'ball_check'], index_col=False)

output_det, output_gt = mod_dataset(input_det, input_gt)

In [ ]:
output_det

In [ ]:
output_path_gt = '/media/alb/Elements/DATASET/SoccerNet/tracking/output/gt/gt.txt'
output_path_det = '/media/alb/Elements/DATASET/SoccerNet/tracking/output/det/det.txt'
output_path_img = '/media/alb/Elements/DATASET/SoccerNet/tracking/output/img1'


out_files = os.listdir(output_path_img)
for o_f in out_files:
    os.remove(output_path_img + '/' + o_f)


files = os.listdir(input_path_img)
for f in files:
    shutil.copy(input_path_img + '/' + f, output_path_img)

output_det.to_csv(output_path_det)
output_gt.to_csv(output_path_gt)